#Transform Orders Data - String to JSON
1. Pre-process the JSON String to fix the Data Quality issues
2. Transform JSON String to JSON object
3. Write transformed data to the Silver schema

In [0]:
%sql
SELECT * FROM gizmobox.bronze.v_orders;

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW tv_orders_fixed
AS
SELECT value,
       regexp_replace(value, '"order_date": (\\d{4}-\\d{2}-\\d{2})', '"order_date": "\$1"') AS fixed_value
FROM gizmobox.bronze.v_orders;

In [0]:
%sql
SELECT schema_of_json(fixed_value) AS schema,
       fixed_value
FROM tv_orders_fixed
LIMIT 1;

In [0]:
%sql
SELECT from_json(fixed_value,
                 'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') AS json_value,
      fixed_value
FROM tv_orders_fixed
LIMIT 1;

In [0]:
%sql
SELECT from_json(fixed_value,
                 'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') AS json_value,
      fixed_value
FROM tv_orders_fixed
LIMIT 1;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gizmobox.silver.orders_json
SELECT from_json(fixed_value,
                 'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') AS json_value
FROM tv_orders_fixed;

In [0]:
%sql 
SELECT * from gizmobox.silver.orders_json;